# LCEL(Langchain Expression Language)
* 사용자의 입력 => 프롬프트 템플릿 => LLM 모델 전달
* 각 단계를 차례로 엮어서 하나의 파이프라인으로 만들어주는 것

chain = prompt | model | output_parser

### 프롬프트 템플릿
* 사용자의 입력 변수를 받아서 완전한 프롬프트 문자열을 만드는데에 사용하는 템플릿

In [4]:
from langchain_core.prompts import PromptTemplate

template = "{lecture}에 대해 설명해주세요."

prompt_template = PromptTemplate(template=template, input_variables=["lecture"])

In [5]:
lecture = input("궁금한 과목을 입력하세요.")

prompt = prompt_template.format(lecture=lecture)

print(prompt)

스프링에 대해 설명해주세요.


In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1
)

In [7]:
chain = prompt_template | model

In [8]:
# Prompt Template의 변수에 작성할 때는 Dictionary 형태로 Key-Value쌍으로 입력을 주어야 한다.
input = {"lecture": "랭체인"}

In [ ]:
chain.invoke(input)

### 출력 형식 지정(Output Parser)
* LLM 모델이 응답해주는 내용에 대한 형식을 지정해주는 것

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [11]:
chain = prompt_template | model | output_parser

In [12]:
chain.invoke(input)

'랭체인은 블록체인 기술을 활용하여 게임 산업에 혁신을 가져오는 플랫폼입니다. 랭체인은 게임 내 아이템의 소유권을 블록체인에 기록하고 거래할 수 있도록 하여 게임 경제를 투명하고 안전하게 만들어줍니다. 또한 랭체인은 게임 개발자와 플레이어 간의 상호작용을 촉진하고 게임 내 경제 생태계를 활성화시키는데 도움을 줍니다. 이를 통해 게임 산업의 성장과 발전을 촉진하며 새로운 비즈니스 모델을 제시하는 역할을 합니다.'

### 영어 회화를 알려주는 선생님 챗봇
### 어떤 상황을 주면 그 상황에 맞는 영어 회화 + 한글 해석 제공받기

In [13]:
template = """
당신은 영어를 가르치는 10년차 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해주세요.
양식은 [FORMAT]을 참고하여 작성해주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿 작성
prompt = prompt_template.from_template(template)

# 모델 객체 작성
model = ChatOpenAI(model_name="gpt-4o")

# 문자열 출력 Parser 객체 생성
output_parser = StrOutputParser()

In [14]:
# 체인 구성
chain = prompt | model | output_parser

In [15]:
# 체인 실행
print(chain.invoke({"question": "저는 식당에 가서 스파게티를 주문하고 싶어요."}))

- 영어 회화:
  - Customer: Hello, I would like to order some spaghetti, please.
  - Waiter: Of course! Would you like any specific type of spaghetti, such as Bolognese or Carbonara?
  - Customer: I'll have the Bolognese, please.
  - Waiter: Great choice. Would you like anything to drink with that?
  - Customer: Yes, I'll have a glass of red wine, please.
  - Waiter: Certainly. Your order will be ready shortly.

- 한글 해석:
  - 손님: 안녕하세요, 스파게티를 주문하고 싶어요.
  - 종업원: 물론입니다! 특정 종류의 스파게티를 원하시나요? 예를 들면 볼로네제나 까르보나라 같은요.
  - 손님: 볼로네제로 주세요.
  - 종업원: 좋은 선택입니다. 함께 드실 음료가 필요하신가요?
  - 손님: 네, 레드 와인 한 잔 주세요.
  - 종업원: 알겠습니다. 주문하신 음식은 곧 준비될 겁니다.


### 실습
* 여러가지 재료를 변수로 입력받고, 재료로 요리할 수 있는 요리의 요리명, 재료, 조리법을 포맷으로 받는 템플릿을 작성하고 문자열 출력 파서로 응답을 받을 수 있게 체인을 구성하세요.

In [17]:
template = """
사용자가 사용할 재료들을 입력하면 그 재료들로 할 수 있는 요리를 [FORMAT]에 맞게 답변하세요.

#재료:
{ingredients}

#FORMAT:
- 요리명:
- 재료:
- 조리법:
"""

prompt = prompt_template.from_template(template)

model = ChatOpenAI(model="gpt-4o")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"ingredients": "밀가루, 토마토, 모짜렐라치즈, 루꼴라, 새우"}))

- 요리명: 새우 피자  
- 재료: 밀가루, 토마토, 모짜렐라치즈, 루꼴라, 새우  
- 조리법:  
  1. **피자 도우 준비**: 밀가루에 물과 소금을 넣고 반죽하여 피자 도우를 만듭니다. 반죽을 둥글게 펴서 준비합니다.  
  2. **토마토 소스 만들기**: 토마토를 잘게 썰어 팬에 넣고 끓여서 소스를 만듭니다. 소금을 약간 첨가하여 간을 맞춥니다.  
  3. **도우에 소스 바르기**: 준비한 피자 도우에 토마토 소스를 고르게 발라줍니다.  
  4. **토핑 추가**: 도우 위에 모짜렐라치즈를 듬뿍 올리고, 새우를 적당히 배치합니다.  
  5. **피자 굽기**: 예열된 오븐에 피자를 넣고 200도에서 15분 정도 굽습니다.  
  6. **마무리**: 구워진 피자 위에 신선한 루꼴라를 올려 마무리합니다.  
  7. **서빙**: 따뜻하게 잘라서 서빙합니다.  
